In [1]:
import numpy as np 
import copy
from scipy.optimize import minimize
from scipy.linalg import sqrtm 
from scipy.stats import poisson

In [ ]:
# ---  Class for the quantum systems ------------------
class QuantumState:
    """|ψ⟩ or ⟨ψ| for a single photon polarisation qubit."""

    _LABELS = {
        'H': np.array([[1], [0]], dtype=complex),                # |H⟩
        'V': np.array([[0], [1]], dtype=complex),                # |V⟩
        'D': 1/np.sqrt(2) * np.array([[1], [1]],  dtype=complex),# |D⟩
        'R': 1/np.sqrt(2) * np.array([[1], [-1j]], dtype=complex),# |R⟩
        'L': 1/np.sqrt(2) * np.array([[1], [1j]],dtype=complex) # |L⟩
    }

    def __init__(self,
                 label: str, 
                 bra: bool=False
                 ):
        
        if label not in self._LABELS:
            raise ValueError("State must be one of H,V,D,R,L")
        
        self._ket = self._LABELS[label].copy()
        self.bra = bra     # flag: True ⇒ row vector

    @property
    def ket(self) -> np.ndarray:
        return self._ket if not self.bra else self._ket.conj().T

    @property
    def bra_vec(self) -> np.ndarray:
        return self._ket.conj().T if not self.bra else self._ket

class TensorState:
    """|ψ_A⟩⊗|ψ_B⟩   (or its dual)"""

    def __init__(self, 
                 psiA: QuantumState, 
                 psiB: QuantumState, 
                 bra: bool=False
                 ):
        
        self._ket = np.kron(psiA.ket, psiB.ket)   # shape (4,1)
        self.bra = bra

    @property
    def ket(self) -> np.ndarray:      # (4,1)
        return self._ket if not self.bra else self._ket.conj().T

    @property
    def bra_vec(self) -> np.ndarray:  # (1,4)
        return self._ket.conj().T if not self.bra else self._ket
# ------------------------------------------------------



# --- Functions for computing rho from linear conversion
def gamma_operator(idx):
 
    matrices = {
        0: 0.5 * np.array([[0,1,0,0],
                           [1,0,0,0],
                           [0,0,0,1],
                           [0,0,1,0]])
                           ,
        1: 0.5 * np.array([[0,-1j,0,0],
                           [1j,0,0,0],
                           [0,0,0,-1j],
                           [0,0,1j,0]])
                           ,
        2: 0.5 * np.array([[1,0,0,0],
                           [0,-1,0,0],
                           [0,0,1,0],
                           [0,0,0,-1]])
                           ,
        3: 0.5 * np.array([[0,0,1,0],
                           [0,0,0,1],
                           [1,0,0,0],
                           [0,1,0,0]])
                           ,
        4: 0.5 * np.array([[0,0,0,1],
                           [0,0,1,0],
                           [0,1,0,0],
                           [1,0,0,0]])
                           ,
        5: 0.5 * np.array([[0,0,0,-1j],
                           [0,0,1j,0],
                           [0,-1j,0,0],
                           [1j,0,0,0]])
                           ,
        6: 0.5 * np.array([[0,0,1,0],
                           [0,0,0,-1],
                           [1,0,0,0],
                           [0,-1,0,0]])
                           ,
        7: 0.5 * np.array([[0,0,-1j,0],
                           [0,0,0,-1j],
                           [1j,0,0,0],
                           [0,1j,0,0]])
                           ,
        8: 0.5 * np.array([[0,0,0,-1j],
                           [0,0,-1j,0],
                           [0,1j,0,0],
                           [1j,0,0,0]])
                           ,
        9: 0.5 * np.array([[0,0,0,-1],
                            [0,0,1,0],
                            [0,1,0,0],
                            [-1,0,0,0]])
                            ,
        10: 0.5 * np.array([[0,0,-1j,0],
                            [0,0,0,1j],
                            [1j,0,0,0],
                            [0,-1j,0,0]])
                            ,
        11: 0.5 * np.array([[1,0,0,0],
                            [0,1,0,0],
                            [0,0,-1,0],
                            [0,0,0,-1]])
                            ,
        12: 0.5 * np.array([[0,1,0,0],
                            [1,0,0,0],
                            [0,0,0,-1],
                            [0,0,-1,0]])
                            ,
        13: 0.5 * np.array([[0,-1j,0,0],
                            [1j,0,0,0],
                            [0,0,0,1j],
                            [0,0,-1j,0]])
                            , 
        14: 0.5 * np.array([[1,0,0,0],
                            [0,-1,0,0],
                            [0,0,-1,0],
                            [0,0,0,1]])
                            , 
        15: 0.5 * np.array([[1,0,0,0],
                            [0,1,0,0],
                            [0,0,1,0],
                            [0,0,0,1]])
    }

    return matrices[idx]

def compute_B_operator() -> np.ndarray:
    """Return B_{νμ} = ⟨ν| Γ_μ |ν⟩  (16×16)."""

    nu_map = {
        0: ['H','H'],  1:['H','V'],  2:['V','V'],  3:['V','H'],
        4: ['R','H'],  5:['R','V'],  6:['D','V'],  7:['D','H'],
        8: ['D','R'],  9:['D','D'], 10:['R','D'], 11:['H','D'],
        12:['V','D'], 13:['V','L'], 14:['H','L'], 15:['R','L']
    }

    B = np.zeros((16,16), dtype=complex)


    kets = {nu: TensorState(QuantumState(nu_map[nu][0]),
                           QuantumState(nu_map[nu][1])).ket
            for nu in range(16)}

    for mu in range(16):
        Gamm = gamma_operator(mu)        # 4×4
        for nu in range(16):
            ket = kets[nu]            # (4,1)
            bra = ket.conj().T       # (1,4)
            B[nu, mu] = (bra @ (Gamm @ ket)).item()

    return B

def compute_M_nu(nu,B_inv):

    M = np.zeros(shape=(16,4,4), dtype='complex')

    for mu in range(16):

        M[mu] = B_inv[mu,nu] * gamma_operator(mu)

    M_nu = np.sum(M, axis=0)
    
    return M_nu

def compute_rho(denom, B_inv, coinc):

    rho = np.zeros(shape = (4,4), dtype='complex')

    for nu in range(16):

        rho += compute_M_nu(nu,B_inv) * coinc[nu]

    return rho / denom
# -----------------------------------------------------



# --- functions to compute rho from MLE ---------------
def first_minor(rho,i,j):

    first = 0

    rho = np.delete(rho, i, axis=0)
    rho = np.delete(rho, j, axis=1)

    first = np.linalg.det(rho)

    return first

def second_minor(rho,i,j,k,l):

    rho = np.delete(rho, (i,k), axis=0)
    rho = np.delete(rho, (j,l), axis=1)

    second = np.linalg.det(rho)

    return second

def computte_T_initial(rho):

    T = np.zeros((4,4), dtype='complex')

    first_00    = first_minor(rho,0,0)
    first_01    = first_minor(rho,0,1)
    second_0011 = second_minor(rho,0,0,1,1)
    second_0112 = second_minor(rho,0,1,1,2)
    second_0012 = second_minor(rho,0,0,1,2)

    T[0,0] = np.sqrt(np.linalg.det(rho) / first_00)
    T[1,0] = first_01 / np.sqrt(first_00 * second_0011)
    T[1,1] = np.sqrt(first_00 / second_0011)
    T[2,0] = second_0112 / np.sqrt(rho[3,3] * second_0011)
    T[2,1] = second_0012 / np.sqrt(rho[3,3] * second_0011)
    T[2,2] = np.sqrt(second_0011 / rho[3,3])
    T[3,0] = rho[3,0] / np.sqrt(rho[3,3]) 
    T[3,1] = rho[3,1] / np.sqrt(rho[3,3]) 
    T[3,2] = rho[3,2] / np.sqrt(rho[3,3]) 
    T[3,3] = np.sqrt(rho[3,3])

    return T

def build_T(t):
    """Restituisce la matrice T tridiagonale a partire da 16 reali."""
    t = np.asarray(t, dtype=float)
    T = np.array([[t[0]              , 0                 , 0                 , 0],
                  [t[4] + 1j*t[5]    , t[1]              , 0                 , 0],
                  [t[10]+ 1j*t[11]   , t[6]+1j*t[7]      , t[2]              , 0],
                  [t[14]+ 1j*t[15]   , t[12]+1j*t[13]    , t[8]+1j*t[9]      , t[3]]],
                 dtype=complex)
    return T

def rho_from_T(T):
    TT = T.conj().T @ T
    return TT / np.trace(TT)

def build_kets():
    nu_map = {
        0:['H','H'],  1:['H','V'],  2:['V','V'],  3:['V','H'],
        4:['R','H'],  5:['R','V'],  6:['D','V'],  7:['D','H'],
        8:['D','R'],  9:['D','D'], 10:['R','D'], 11:['H','D'],
       12:['V','D'], 13:['V','L'], 14:['H','L'], 15:['R','L']
    }
    return {nu: TensorState(QuantumState(a), QuantumState(b)).ket
            for nu,(a,b) in nu_map.items()}

def T_to_vec(T):
    """Converte la tua T tridiagonale (4×4) in 16 parametri reali."""
    v = np.zeros(16, dtype=float)
    v[0]  = T[0,0].real
    v[1]  = T[1,1].real
    v[2]  = T[2,2].real
    v[3]  = T[3,3].real
    v[4]  = T[1,0].real; v[5]  = T[1,0].imag
    v[6]  = T[2,1].real; v[7]  = T[2,1].imag
    v[8]  = T[3,2].real; v[9]  = T[3,2].imag
    v[10] = T[2,0].real; v[11] = T[2,0].imag
    v[12] = T[3,1].real; v[13] = T[3,1].imag
    v[14] = T[3,0].real; v[15] = T[3,0].imag
    return v

def likelihood(t, counts, N, kets, floor=1e-12):

    T   = build_T(t)
    rho = rho_from_T(T)
    L   = 0.0

    for nu in range(16):

        sn = np.real((kets[nu].conj().T @ rho @ kets[nu]).item())
        n̄  = N * sn
        L += (n̄ - counts[nu])**2 / (2.0 * n̄)

    return L
# ---------------------------------------------------



# --- functions to compute Entanglement statistics --
def compute_VonNeumannEntropy(rho, B_inv, coinc):

    N = coinc[0]+coinc[1]+coinc[2]+coinc[3]
    eigvals, eigvec = np.linalg.eig(rho)
    eigvals = eigvals.real
    entropy = 0
    var = 0
    

    for eval in eigvals:
        entropy -= eval * np.log2(eval)

    for nu in range(16):

        M = compute_M_nu(nu, B_inv)
        e = 0

        for eval, evec in zip(eigvals, eigvec):

            evec = evec.reshape(4,1)
            e -= evec.conj().T @ M @ evec * (1+np.log(eval))/np.log(2)


        var += e**2 * (coinc[nu]/N**2)

    return entropy.real

def compute_fidelity(rho):

    density_phi_minus = 0.5 * np.array([[1,0,0,-1],
                                        [0,0,0,0 ],
                                        [0,0,0,0 ],
                                        [-1,0,0,1]])
    sigma = density_phi_minus
    
    sqrt_rho = sqrtm(rho)
    mat = sqrt_rho @ sigma @ sqrt_rho
    sqrt_mat = sqrtm(mat)
    fidelity = (np.trace(sqrt_mat).real) **2 
 
    return fidelity

def compute_concurrence(rho):
   
    sy = np.array([[0, -1j], [1j, 0]])
    S  = np.kron(sy, sy)
    rho_tilde = (S @ rho.conj() @ S)

    R = np.array(sqrtm(sqrtm(rho) @ rho_tilde @ sqrtm(rho)),dtype='complex')
    lambdas = np.sort(np.linalg.eig(R)[0])
    C = max(0, np.sum(lambdas[3]-
                      lambdas[2]-
                      lambdas[1]-
                      lambdas[0]))
    return C.real
# ---------------------------------------------------

In [18]:
def routine_quantum_tomography():

    # collecting data
    coinc =       {0 : 3831,
                   1 : 45,
                   2 : 3655,
                   3 : 71,
                   4 : 2218,
                   5 : 1607,
                   6 : 1577,
                   7 : 2219,
                   8 : 2305,
                   9 : 408,
                   10: 2444,
                   11: 1934,
                   12: 1588,
                   13: 1493,
                   14: 2102,
                   15: 370}
    
    coinc_paper = {0: 34749,
                   1: 324,
                   2: 35805,
                   3: 444,
                   4: 16324,
                   5: 17521,
                   6: 13441,
                   7: 16901,
                   8: 17932,
                   9: 32028,
                   10:15132,
                   11:17238,
                   12:13171,
                   13:17170,
                   14:16722,
                   15:33586
                   }

    # normalization factor 
    N = coinc[0] + coinc[1] + coinc[2] + coinc[3]

    B_inv = np.linalg.inv(compute_B_operator())
    rho = compute_rho(N, B_inv, coinc)

    print('-------------density matrix from linear inversion------------ ')
    print('ρ_LI =')
    print(np.round(rho,4), '\n')

    print('Hermitianity ρ†-ρ = 0')
    print(np.round(rho.conj().T - rho,3))

    print('\n Normalization Tr(ρ) = 1 ')
    print('Tr(ρ) = ',np.trace(rho))

    print('\n positivity Eigenval(ρ) >= 0  ')
    print('Eigenval: ', np.round(np.linalg.eig(rho)[0].real,3))





    print('\n \n \n -------------density matrix from MLE ------------ ')

    T_initial = computte_T_initial(rho)
    kets = build_kets()
    counts = np.array([coinc[nu] for nu in range(16)], dtype=float)

    t0 = T_to_vec(T_initial)

    opt = minimize(likelihood, t0,
                   args=(counts, N, kets),
                   method='Powell',
                   options={'xtol':1e-10, 'ftol':1e-10, 'maxiter':6000})

    t_opt  = opt.x
    rho_ml = rho_from_T(build_T(t_opt))

    # ---------- 9. Output risultati ----------
    print("ρ_ML =")
    np.set_printoptions(suppress=True,precision=4)
    print(rho_ml, '\n')

    print('Hermitianity ρ†-ρ = 0')
    print(np.round(rho_ml.conj().T - rho_ml,3))

    print('\n Normalization Tr(ρ) = 1 ')
    print('Tr(ρ) = ',np.trace(rho_ml))

    print('\n positivity Eigenval(ρ) >= 0  ')
    print('Eigenval: ', np.round(np.linalg.eig(rho_ml)[0].real,3))


    
    print('\n \n \n ------------- Entanglement Statistics------------ ')
    entropy = compute_VonNeumannEntropy(rho_ml,B_inv,coinc)
    fidelity = compute_fidelity(rho_ml)
    concurrence = compute_concurrence(rho_ml)
    print(f'Von Neuman entropy for ρ_ML: {np.round(entropy,3)}')
    print(f'Fidelity with |Φ⁻⟩: {np.round(fidelity,3)}')
    print(f'Concurrence for ρ_ML: {np.round(concurrence,3)}')


In [19]:
routine_quantum_tomography()

-------------density matrix from linear inversion------------ 
ρ_LI =
[[ 0.5039+0.j     -0.0005-0.0216j  0.0353+0.0351j -0.367 +0.1283j]
 [-0.0005+0.0216j  0.0059+0.j      0.0117+0.0483j -0.0359-0.032j ]
 [ 0.0353-0.0351j  0.0117-0.0483j  0.0093+0.j     -0.0362+0.0487j]
 [-0.367 -0.1283j -0.0359+0.032j  -0.0362-0.0487j  0.4808+0.j    ]] 

Hermitianity ρ†-ρ = 0
[[0.-0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.-0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.-0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-0.j]]

 Normalization Tr(ρ) = 1 
Tr(ρ) =  (1+0j)

 positivity Eigenval(ρ) >= 0  
Eigenval:  [ 0.891  0.113  0.04  -0.044]

 
 
 -------------density matrix from MLE ------------ 
ρ_ML =
[[ 0.5042+0.j      0.0122-0.0231j  0.0329+0.0352j -0.3792+0.1196j]
 [ 0.0122+0.0231j  0.0062+0.j      0.0006+0.0064j -0.035 -0.0292j]
 [ 0.0329-0.0352j  0.0006-0.0064j  0.0096+0.j     -0.0281+0.0477j]
 [-0.3792-0.1196j -0.035 +0.0292j -0.0281-0.0477j  0.48  +0.j    ]] 

Hermitianity ρ†-ρ = 0
[[0.-0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j

In [5]:
def montecarlo_simulation(n_steps,coinc):

    ent = np.zeros(n_steps)
    fid = np.zeros(n_steps)
    con = np.zeros(n_steps)

    for n in range(n_steps):
        #if n%10 == 0:
        print('step: ',n)
        
        new_coinc = {0: poisson.rvs(coinc[0],size = 1),
                     1: poisson.rvs(coinc[1],size = 1),
                     2: poisson.rvs(coinc[2],size = 1),
                     3: poisson.rvs(coinc[3],size = 1),
                     4: poisson.rvs(coinc[4],size = 1),
                     5: poisson.rvs(coinc[5],size = 1),
                     6: poisson.rvs(coinc[6],size = 1),
                     7: poisson.rvs(coinc[7],size = 1),
                     8: poisson.rvs(coinc[8],size = 1),
                     9: poisson.rvs(coinc[9],size = 1),
                     10: poisson.rvs(coinc[10],size = 1),
                     11: poisson.rvs(coinc[11],size = 1),
                     12: poisson.rvs(coinc[12],size = 1),
                     13: poisson.rvs(coinc[13],size = 1),
                     14: poisson.rvs(coinc[14],size = 1),
                     15: poisson.rvs(coinc[15],size = 1)
                    
                    }

        # normalization factor 
        N = new_coinc[0] + new_coinc[1] + new_coinc[2] + new_coinc[3]

        B_inv = np.linalg.inv(compute_B_operator())
        rho = compute_rho(N, B_inv, new_coinc)


        T_initial = computte_T_initial(rho)
        kets = build_kets()
        counts = np.array([new_coinc[nu] for nu in range(16)], dtype=float)

        t0 = T_to_vec(T_initial)

        opt = minimize(likelihood, t0,
                       args=(counts, N, kets),
                       method='Powell',
                       options={'xtol':1e-10, 'ftol':1e-10, 'maxiter':6000})

        t_opt  = opt.x
        rho_ml = rho_from_T(build_T(t_opt))

        entropy = compute_VonNeumannEntropy(rho_ml,B_inv,coinc)
        fidelity = compute_fidelity(rho_ml)
        concurrence = compute_concurrence(rho_ml)

        ent[n] = entropy
        fid[n] = fidelity
        con[n] = concurrence
    
    print(ent,'\n')
    print(fid,'\n')
    print(con,'\n')

    ent = np.delete(ent,np.isnan(ent)) 
    fid = np.delete(fid,np.isnan(fid)) 
    con = np.delete(con,np.isnan(con)) 

    return [ent,fid,con]


        
        



    

In [ ]:
coinc =       {0 : 3831,
                   1 : 45,
                   2 : 3655,
                   3 : 71,
                   4 : 2218,
                   5 : 1607,
                   6 : 1577,
                   7 : 2219,
                   8 : 2305,
                   9 : 408,
                   10: 2444,
                   11: 1934,
                   12: 1588,
                   13: 1493,
                   14: 2102,
                   15: 370}

sim = montecarlo_simulation(100,coinc)

In [ ]:
print(np.round(np.mean(sim[0]),2))
print(np.round(np.std(sim[0]),2))

print('\n',np.round(np.mean(sim[1]),2))
print(np.round(np.std(sim[1]),2))

print('\n',np.round(np.mean(sim[2]),2))
print(np.round(np.std(sim[2]),2))

0.49
0.03

 0.87
0.01

 0.8
0.02


In [ ]:
val = [0.50081434, 0.51013223, 0.54087751, 0.45292374, 0.46969425, 0.48273471,
 0.49772241, 0.50066818, 0.45465613, 0.48770228, 0.48331308, 0.40966101,
 0.46097371, 0.48148445, 0.45699607, 0.43850943, 0.42467092, 0.53424544,
 0.48272156, 0.53123808, 0.45909503, 0.49269328, 0.43640528, 0.47478068,
 0.45760405, 0.46653397, 0.51010053, 0.54060144, 0.44994632, 0.53072926,
 0.49439397, 0.45635055, 0.44566823, 0.53652069, 0.45015327, 0.49518988,
 0.47476904, 0.49594258, 0.48711934, 0.48851092, 0.49535148, 0.51672577,
 0.49348909, 0.47286368, 0.51483026, 0.41012057, 0.54095706, 0.47641979,
 0.49724052, 0.48671954, 0.52064102, 0.47460835, 0.50230059,
 0.5333855 , 0.48920742, 0.48810031, 0.46297773, 0.544642  , 0.48263901,
 0.50474727, 0.4956346 , 0.5149839 , 0.45074763,    
 0.49665197, 0.46457412, 0.51647133, 0.54461702, 0.53040932, 0.46932976,
 0.5135872 , 0.50173917, 0.52443482, 0.51130857, 0.4465135 , 0.50876571,
 0.41836036, 0.51893745, 0.48914074, 0.49925475, 0.50541107, 0.49590068,
 0.44670154, 0.41975843, 0.49488286, 0.4579162 , 0.49639702, 0.50162527,
 0.57550147, 0.47178735, 0.52837073, 0.49677058, 0.53790132, 0.56887358,
 0.47342331, 0.48544792, 0.45773562, 0.51383695] 

In [ ]:
np.std(val)

0.03364694665708683